In [1]:
import pandas as pd
import numpy as np
import os


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import spacy
from textblob import TextBlob



# Model 1: NLI
 - TODO: figure out ground truth retrieval/knowledge graph
 - considering using [k-BERT](https://arxiv.org/pdf/1909.07606.pdf)

# Model 2: political Bias
- use rbf svm with tf-idf
- outputs: one class label, as an int

In [2]:
df = pd.read_csv('liar_plus/train2.tsv', delimiter='\t', header = None)
df = df.drop(columns = [0])


df.rename({1: 'id', 2: 'label', 3: 'statement', 4: 'subject', 5: 'speaker', 6: 'job-title',
           7: 'state_info', 8: 'party_affiliation', 9: 'barely_true_counts', 10: 'false_counts',
           11: 'half_true_counts', 12: 'mostly_true_counts', 13: 'pants_on_fire_counts', 14: 'context',
           15: 'justification'
          }, axis = 1, inplace = True)

df = df[~df['statement'].isna()]


uninformative = {'organization', 'newsmaker', 'activist', 'state-official', 'government-body',
'journalist', 'columnist', 'talk-show-host', 'education-official', 'business-leader', 
 'Moderate', 'democratic-farmer-labor', 'ocean-state-tea-party-action' }

df_bias = df[~df['party_affiliation'].isin(uninformative)]
df_bias = df_bias[~df_bias['party_affiliation'].isna()]

In [3]:
to_replace = {
    'constitution-party': 0, 'libertarian': 1,
    'tea-party-member': 2,
    'republican': 3, 'none': 4, 'independent': 5,
    'liberal-party-canada': 6, 'labor-leader': 7, 
    'democrat': 8, 'green': 9
}

df_bias['party_affiliation'] = df_bias['party_affiliation'].replace(to_replace)

In [4]:
X, y = df_bias['statement'], df_bias['party_affiliation']


X_train, X_test, y_train, y_test = (
    train_test_split(X, y, test_size=.2)
)

tfidf_bias = TfidfVectorizer()
X_train = tfidf_bias.fit_transform(X_train)

X_test = tfidf_bias.transform(X_test)


In [5]:
bias_model = SVC(gamma=2, C=1)
bias_model.fit(X_train, y_train)

SVC(C=1, gamma=2)

# Model 3: Fallacies
- outputs: one t/f label 
- TODO: turn to multiclass classification with specific logical fallacies

In [6]:
binary_fp = os.path.join(os.pardir, "fallacy_detection", "cleaned_binary.csv")


In [7]:
binary = pd.read_csv(binary_fp)
binary.replace({'Invalid': 0, 'Valid': 1}, inplace = True)


In [8]:
X, y = binary['input'], binary['label']


X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=.2)

tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(X_train)

In [9]:
fallacy_model = AdaBoostClassifier()
fallacy_model.fit(X_train, y_train)

AdaBoostClassifier()

In [10]:
(fallacy_model.predict(tfidf.transform(X_test)) == y_test).mean()

0.7125

# "Model" 4: NER + sentiment
- Extract named entities, to be passed in to an embedding layer in final model
- Compute sentiment score

In [11]:
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

Defaulting to user installation because normal site-packages is not writeable
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
#df = df.drop(index = 1280)

In [13]:
def get_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    return entities

df['entities'] = df['statement'].apply(get_entities)

In [14]:
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

df['sentiment'] = df['statement'].apply(get_sentiment)


# Meta-Model: feedforward NN
- Takes in the outputs of the previous models and outputs a single scalar representing the overall level of misinformation
- The response variable (misinformation score) is simply the politifact rating mapped to integers 0-5
- TODO: need to tune hyperparameters more.

In [15]:
def vectorize(text):
    
    #bias
    vec_text = tfidf_bias.transform([text])
    bias = bias_model.predict(vec_text)[0]
    
    #fallacies
    fall_vec = tfidf.transform([text])
    fallacy = fallacy_model.predict(fall_vec)[0]
    
    #sentiment
    sentiment = get_sentiment(text)
    
    entities = get_entities(text)
    
    
    return np.array([bias, fallacy, sentiment,
                     entities], dtype = 'object')

In [17]:
class MetaModel(nn.Module):
    def __init__(self, entity_embedding_dim, vocab_size):
        super(MetaModel, self).__init__()

        # Embedding layer for named entities
        self.entity_embedding = nn.EmbeddingBag(vocab_size, 
                                                entity_embedding_dim,
                                                sparse=True)  # Assuming a maximum of 10000 unique entities

        # Fully connected layers for scalar inputs
        self.fc_scalar1 = nn.Linear(3, 15)
        self.fc_scalar2 = nn.Linear(15, 30)

        # Fully connected layers for the combined features
        self.fc_combined1 = nn.Linear(entity_embedding_dim + 30, 20)
        self.fc_combined2 = nn.Linear(20, 1)

    def forward(self, scalar_inputs, entity_input):
        
        x_entity = self.entity_embedding(entity_input)

        x_scalar = F.relu(self.fc_scalar1(scalar_inputs))
        x_scalar = F.relu(self.fc_scalar2(x_scalar))

        # Combine scalar and entity features
        x_combined = torch.cat((x_entity, x_scalar), dim=1)

        x_combined = F.relu(self.fc_combined1(x_combined))
        x_combined = self.fc_combined2(x_combined)

        return x_combined

**Data Preparation**

In [16]:
label_map = {'pants-fire': 5, 'false': 4, 'half-true': 3, 
             'barely-true': 2, 'mostly-true': 1, 'true': 0}

liar_rating = df['label'].replace(label_map)

**Training**

In [17]:
entity_cats = len(['CARDINAL', 'DATE', 'EVENT', 
'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 
 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART'])

In [18]:
vector_statement = df['statement'].apply(vectorize)

In [19]:
X_train, X_test, y_train, y_test = \
    train_test_split(vector_statement, liar_rating, test_size=.2)

In [160]:
X_train_numerical = torch.tensor(X_train.str[:3].values.tolist(), dtype=torch.float32)

vocab_size = 10000

entities = X_train.str[-1].values.tolist()

entity_to_index_mapping = {entity: idx + 1 for idx, 
                           entity in enumerate(set(entity for entity_list in entities for entity in entity_list))}

entity_inputs = pad_sequence([torch.tensor([entity_to_index_mapping.get(word, 0) for 
                                            word in entity_list]) 
                              for entity_list in entities], batch_first=True)


y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)

model = MetaModel(entity_embedding_dim=5, vocab_size=vocab_size)

criterion = nn.MSELoss()

# Separate parameters for the embedding layer and other parameters
embedding_params = list(model.entity_embedding.parameters())
other_params = [param for name, param in model.named_parameters() if
                not any(embedding_name in name for embedding_name in ["entity_embedding"])]

# Separate optimizers for each set of parameters
optimizer_embedding = optim.SparseAdam(embedding_params, lr=2e-1)
optimizer_other = optim.Adam(other_params, lr=2e-3)

train_data = TensorDataset(X_train_numerical, entity_inputs, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for numerical_inputs, entities, labels in train_loader:
        optimizer_embedding.zero_grad()
        optimizer_other.zero_grad()

        # Forward pass
        outputs = model(numerical_inputs, entities)

        # Calculate the loss
        loss = criterion(outputs.squeeze(), labels)

        # Backward pass and optimization
        loss.backward()

        # Update embedding parameters
        optimizer_embedding.step()

        # Update other parameters
        optimizer_other.step()

        total_loss += loss.item()

    # Print the average loss for the epoch
    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}')


Epoch 1/5, Loss: 2.7415
Epoch 2/5, Loss: 2.4176
Epoch 3/5, Loss: 2.3896
Epoch 4/5, Loss: 2.3482
Epoch 5/5, Loss: 2.2729


In [161]:
X_test_numerical = torch.tensor(X_test.str[:3].values.tolist(), dtype=torch.float32)

entities_test = X_test.str[-1].values.tolist()


entity_inputs_test = pad_sequence([torch.tensor([entity_to_index_mapping.get(word, 0) for 
                                            word in entity_list]) 
                              for entity_list in entities_test], batch_first=True)

In [162]:
#Predictions

model.eval()
with torch.no_grad():
    predicted_values = model(X_test_numerical, entity_inputs_test).squeeze().numpy()

In [163]:
mean_squared_error(predicted_values, y_test)

2.4638700959097575

In [154]:
#not good at predicting the "very false" labels! no 4s or 5s
(predicted_values >= 4).sum()

0

In [150]:
(y_test >= 4).sum()

590

## Issue with fallacies 
- Not all claims have a logical structure, so it doesn't make sense to use fallacy classification for all of them
- Probably should use in conjunction with a veracity prediction, not as a part of it

In [116]:
fallacy_liar = (fallacy_model.predict(tfidf.transform(df['statement'])))

In [118]:
fallacy_liar_df = df.copy()
fallacy_liar_df['fallacy'] = fallacy_liar

In [139]:
#good example: logical structure which is (arguably) fallacious
print("statement: " + fallacy_liar_df[fallacy_liar_df['fallacy'] == 1]['statement'].iloc[2] + '\n') 

print( 
    "justification: " + fallacy_liar_df[fallacy_liar_df['fallacy'] == 1]['justification'].iloc[2] + '\n'
)

print( "label: " + fallacy_liar_df[fallacy_liar_df['fallacy'] == 1]['label'].iloc[2])


statement: If you look at states that are right to work, they constantly do not have budget deficits and they have very good business climates.

justification: They did generally rank right-to-work states higher than union states, but they considered many other factors, including tax policy, that could account for the better rankings. (Get updates from PolitiFactRI on Twitter.

label: barely-true


In [144]:
#bad example: no logical structure, simple claim
print("statement: " + fallacy_liar_df[fallacy_liar_df['fallacy'] == 1]['statement'].iloc[1] + '\n') 

print( 
    "justification: " + fallacy_liar_df[fallacy_liar_df['fallacy'] == 1]['justification'].iloc[1] + '\n'
)

print( "label: " + fallacy_liar_df[fallacy_liar_df['fallacy'] == 1]['label'].iloc[1])


statement: ISIS supporter tweeted at 10:34 a.m. Shooting began at 10:45 a.m. in Chattanooga, Tenn.

justification: Geller updated the original post, but it did little to stuff the rumor back into the bag.

label: false
